# Dexter GPT-2 Training
First, connect to GPU runtime (Runtime -> Change runtime type -> GPU)

In [ ]:
# Install required packages
!pip install torch transformers datasets

In [ ]:
# Verify GPU
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
# Load required libraries
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import json

In [ ]:
# Mount Google Drive - if using Drive storage
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
# Load training data
# Update path based on your file location
with open('gpt2_training_data.json', 'r') as f:
    training_data = json.load(f)

# Save formatted texts
with open('train.txt', 'w') as f:
    for text in training_data['formatted_texts']:
        f.write(text + '\n')

print(f"Loaded {len(training_data['formatted_texts'])} training samples")

In [ ]:
# Create dataset
def load_dataset(train_path, tokenizer):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128
    )
    return dataset

train_dataset = load_dataset('train.txt', tokenizer)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./dexter-gpt2",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [ ]:
# Train model
trainer.train()

In [ ]:
# Save the model
trainer.save_model()

In [ ]:
# Test generation function
def generate_response(context, model, tokenizer):
    input_text = f"Context: {context} Response:"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(
        inputs['input_ids'],
        max_length=100,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with sample
test_context = "DEBRA: What are you thinking about?"
response = generate_response(test_context, model, tokenizer)
print(f"Context: {test_context}")
print(f"Generated: {response}")